# **Ejercicio 3: Integración Pandas y Spark con datos de películas y críticas **

1.Crear un DataFrame en Pandas con información básica de películas (ID, título, año).

2.Crear un DataFrame en Spark con críticas y puntuaciones de películas.

3.Unir ambos datasets utilizando el ID de la película como llave.

4.Calcular el promedio de puntuación por película y mostrar junto al título y año de estreno.

Instalar el paquete Findspark

In [1]:
 #pip install findspark

Importar las librerías y crear una sesión de Spark

In [2]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark
import pandas as pd
import time
from datetime import datetime
spark = SparkSession.builder.appName("prueba3").getOrCreate()
print(f"Versión de Python: {sys.version}")
print(f"Se cargaron las librerias {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Versión de Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Se cargaron las librerias 2025-09-06 04:44:23


In [3]:
findspark.find()

'/usr/local/lib/python3.12/dist-packages/pyspark'

Convertir datos de pandas a un DataFrame de Spark

In [4]:

datos_peliculas = pd.DataFrame({
    "ID": [1,2,3,4],
    "Titulo": ["Película1","Película2","Película3","Película4"],
    "Año": [2020,2019,2021,2018]
})

peliculas_spark = spark.createDataFrame(datos_peliculas)

Convertir un DataFrame de Spark a pandas únicamente para visualizarlo en un formato tabular con diseño.

In [6]:
peliculas_spark.toPandas()

,ID,Titulo,Año
0,1,Película1,2020
1,2,Película2,2019
2,3,Película3,2021
3,4,Película4,2018


Combinar los datos de críticas con los datos de las películas según su ID.

In [7]:
criticas = spark.createDataFrame([
    Row(PeliculaID=1, Critico="Critico1", Puntuacion=4.5),
    Row(PeliculaID=2, Critico="Critico2", Puntuacion=3.8),
    Row(PeliculaID=3, Critico="Critico1", Puntuacion=4.2),
    Row(PeliculaID=4, Critico="Critico3", Puntuacion=4.7),
])

#Unión
resultado = criticas.groupBy("PeliculaID").agg(avg("Puntuacion").alias("Promedio")) \
    .join(peliculas_spark, criticas.PeliculaID == peliculas_spark.ID, "inner") \
    .select("Titulo", "Año", "Promedio")\
    .orderBy("Titulo")




Convertir el resultado de DataFrame de Spark a pandas únicamente para visualizarlo en un formato tabular con diseño.

In [8]:
resultado.toPandas()

,Titulo,Año,Promedio
0,Película1,2020,4.5
1,Película2,2019,3.8
2,Película3,2021,4.2
3,Película4,2018,4.7
